In [ ]:
from src.document import Document
from pathlib import Path
import random

data_dir = Path("data/processed")
files = list(data_dir.glob("*.json"))
random_file = random.choice(files)
document = Document.load(random_file)

In [ ]:
for sentence in document.sentences[:5]:
    print(sentence)

query the index for the most relevant documents.

In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch()

In [ ]:
index_name = "documents"
search_terms = "evil"
res = es.search(
    index=index_name,
    query={
        "multi_match": {
            "query": search_terms,
            "fields": ["document.title", "page.text"],
        }
    },
    size=1000,
    collapse={"field": "document.id", "inner_hits": {"name": "hits", "size": 5}},
)
res

use concepts to build a simple classifier of concepts in documents

In [4]:
from src.classifiers import RegexClassifier
from src.document import Document
from src.concept import Concept
import json
from pathlib import Path
import random

In [5]:
data_dir = Path("data/processed")
files = list(data_dir.glob("*.json"))
random_file = random.choice(files)
document = Document.load(random_file)

In [6]:
with open("data/concepts.json") as f:
    concepts = json.load(f)

concepts = [Concept.from_dict(concept) for concept in concepts]
concepts

[Concept(ageism)]

In [7]:
classifiers = [RegexClassifier(concept) for concept in concepts]
classifier = classifiers[0]

In [13]:
Document.concept_spans = classifier.predict(document.text)

In [14]:
from rich.progress import track

data_dir = Path("data/processed")
files = list(data_dir.glob("*.json"))
random_file = random.choice(files)
document = Document.load(random_file)